<a href="https://colab.research.google.com/github/Geriys/VKirillV/blob/main/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D0%B0_%D0%BA_%D0%B7%D0%B0%D0%BD%D1%8F%D1%82%D0%B8%D1%8E_4.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Практика
Используемые библиотеки

In [ ]:
import pandas as pd
import numpy as np
import requests # for web-download
import io # for web-download
import re # for data processing


## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


In [ ]:
###
'''
headers={
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0',
}
def get_content(url):
    with requests.Session() as req:
        req.headers.update(headers)
        r = req.get(url).content
    return r

url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
s = get_content(url)
df=pd.read_csv(io.StringIO(s.decode('UTF8')))
###
'''

### Если не работает загрузка on-line
df=pd.read_csv("Химический анализ родника в Нескучном саду.csv", sep=';')
display(df)

,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,б/цвета,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
6,Нитраты (по NO3),мг/дм3,24,не более 45
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5"
8,Хлориды (Cl),мг/дм3,200,не более 350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500


In [ ]:
#Смотрим количество строк и столбцов
df.shape

(14, 4)

In [ ]:
#Смотрим названия столбцов
df.columns

Index(['Показатель', 'Единица измерений', 'Результат анализа', 'Норматив'], dtype='object')

In [ ]:
#Мои черновые практики (можно не смотреть)
'''
print(df['Результат анализа'].values[1])
print(df['Норматив'].values[1])
print(type(df['Норматив'].values[1]))
a = re.findall(r'\d+', df['Норматив'].values[1])
print (a)
a1 = max(re.findall(r'\d+', df['Норматив'].values[1]))
print (a1)

print(df['Норматив'].values[1].startswith(('в')))
print(df['Норматив'].values[1].startswith(('не')))
'''

1
не более 2-3
<class 'str'>
['2', '3']
3
False
True


In [ ]:
# Ваше решение

In [ ]:
def parse_standard(row):
    standard = row['Норматив'].lower().replace(',', '.')
    arr_digits = re.findall(r'\d+(?:\.\d+)?', standard)
    arr_digits = list(map(float, arr_digits))
    try:
        value = float(row['Результат анализа'])
    except:
        return False
    if 'в пределах' in standard:
        min_value = min(arr_digits)
        max_value = max(arr_digits)
        return (value >= min_value) and (value <= max_value)
    if 'не более' in standard:
        max_value = max(arr_digits)
       #  print(value)
      #   print(max_value)
        return value <= max_value
    if 'не менее' in standard:
        min_value = min(arr_digits)
        return value >= min_value

    return False

df['Выводы'] = df.apply(parse_standard, axis=1)


In [ ]:
display(df.set_index('Показатель'))

,Единица измерений,Результат анализа,Норматив,Выводы
Показатель,,,,
pH,единицы pH,8.4,в пределах 6-9,True
Запах,баллы,1,не более 2-3,True
Цветность,градусы,б/цвета,не более 30,False
Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10,True
Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5",True
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",True
Нитраты (по NO3),мг/дм3,24,не более 45,True
Фосфаты (P),мг/дм3,0.36,"не более 3,5",True
Хлориды (Cl),мг/дм3,200,не более 350,True


## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?


In [ ]:
# Ваше решение
print(5/9 * 4/8 * 3/7)

0.11904761904761905


In [ ]:
from random import sample

arr = [0]*5 + [1]*4 #наш массив где 0 это апельсины, а 1 это яблоки
n = 1150000

df = pd.DataFrame({"Выбраны 3 апельсина":list(map(lambda a: sum(sample(arr,3))==0, range(n)))})
print(df.head())

# вероятность
df['Выбраны 3 апельсина'].mean()

   Выбраны 3 апельсина
0                 True
1                False
2                False
3                False
4                False


0.11916782608695652

### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?


В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [ ]:
# Ваше решение
print(3/10 * 7/9)
print(7/30)

0.23333333333333334
0.23333333333333334


In [ ]:
from random import sample

arr1 = [0]*3 + [1]*7 #Ящик со всеми деталями
arr2 = [0]*2 + [1]*7 #Ящик без одной нестандартной детали
n1 = 1500000

df = pd.DataFrame({"Выбраны две детали":list(map(lambda a: sum(sample(arr1,1))==0 and sum(sample(arr2,1))==1, range(n1)))})
print(df.head())

# вероятность
df['Выбраны две детали'].mean()

   Выбраны две детали
0               False
1               False
2               False
3               False
4               False


0.23334